In [9]:
import numpy as np 
import cv2
import pandas as pd
import os 
import pickle

# Face Detection Model
* Load the models

In [3]:
face_detection_model = '../models/res10_300x300_ssd_iter_140000.caffemodel'
face_detection_proto = '../models/deploy.prototxt.txt'
face_descriptor = '../models/openface.nn4.small2.v1.t7'

# Load models using Cv2 dnn
detector_model = cv2.dnn.readNetFromCaffe(face_detection_proto,face_detection_model)
descriptor_model = cv2.dnn.readNetFromTorch(face_descriptor)

In [10]:
!dir E:\programacion\Face_Recognition\celebrity_images

 Volume in drive E is New Volume
 Volume Serial Number is 8EED-9B3C

 Directory of E:\programacion\Face_Recognition\celebrity_images

08/01/2024  01:40 PM    <DIR>          .
08/05/2024  01:10 PM    <DIR>          ..
07/31/2024  09:08 PM    <DIR>          Aamir Khan
07/31/2024  09:08 PM    <DIR>          Angelina Jolie
07/31/2024  09:08 PM    <DIR>          Barack Obama
07/31/2024  09:09 PM    <DIR>          Cristiano Ronaldo
07/31/2024  09:09 PM    <DIR>          Donald Trump
07/31/2024  09:09 PM    <DIR>          Elon Musk
07/31/2024  09:09 PM    <DIR>          Joe Biden
07/31/2024  09:09 PM    <DIR>          Leonardo DiCaprio
07/31/2024  09:09 PM    <DIR>          Lionel Messi
07/31/2024  09:09 PM    <DIR>          Robert Downey Jr
07/31/2024  09:09 PM    <DIR>          Roger Federer
07/31/2024  09:09 PM    <DIR>          Sachin Tendulkar
07/31/2024  09:09 PM    <DIR>          Salman Khan
07/31/2024  09:09 PM    <DIR>          Scarlett Johansson
07/31/2024  09:09 PM    <DIR>        

In [11]:
# consider sample image
img = cv2.imread('../celebrity_images/Sachin Tendulkar/2200.jpg')
cv2.imshow('sample',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [12]:
def helper(image_path):  
    img =cv2.imread(image_path)
    # Step-1: fdce detection
    image = img.copy()
    h,w = image.shape[:2]
    img_blob = cv2.dnn.blobFromImage(image,1,(300,300),(104,177,123),swapRB=False,crop=False)
    # set the input
    detector_model.setInput(img_blob)
    detections = detector_model.forward()

    if len(detections) > 0:
        i = np.argmax(detections[0,0,:,2])# consider the face with max confidence score
        confidence = detections[0,0,i,2]
        if confidence > 0.5:
            box = detections[0,0,i,3:7]*np.array([w,h,w,h])
            (startx,starty,endx,endy) = box.astype('int')
            # step-2: feature Extraction or Embedding
            roi = image[starty:endy,startx:endx].copy()
            # get the face descriptors
            faceblob = cv2.dnn.blobFromImage(roi,1/255,(96,96),(0,0,0),swapRB=True,crop=True)
            descriptor_model.setInput(faceblob)
            vectors = descriptor_model.forward()
            
            return vectors
    return None
        



In [13]:
helper('../celebrity_images/Sachin Tendulkar/2200.jpg')

array([[-0.04844675,  0.0797378 , -0.09963042,  0.05219206,  0.02248179,
         0.2019831 ,  0.13852584, -0.06266056, -0.06219403,  0.01123158,
         0.00230624,  0.07717839,  0.01544621, -0.03280809, -0.0198784 ,
        -0.10874753, -0.09319475, -0.01268351,  0.10149096, -0.03454285,
         0.03998316, -0.1423884 , -0.11665355, -0.01645252,  0.07435048,
         0.0027732 , -0.17686342, -0.14906752,  0.00644351,  0.10620246,
         0.08508852,  0.08288774, -0.08299836, -0.03749985,  0.04876523,
         0.05993139, -0.07223046, -0.06693252,  0.04339122, -0.07137279,
         0.14999197, -0.0811222 ,  0.04310588,  0.04121743, -0.09240641,
        -0.03493488,  0.11330164, -0.07407272, -0.12562124, -0.14079265,
        -0.07809558, -0.01990941,  0.07394507, -0.03580351,  0.08483166,
         0.13799942, -0.11584078,  0.19711755, -0.08613208, -0.01615908,
        -0.10057899,  0.07192262,  0.20695503, -0.21651323,  0.16872875,
         0.14802314,  0.04371178, -0.0987717 , -0.1

# apply helper function to all images and get face descriptors

In [8]:
data = dict(data=[],label=[])

In [9]:
folders = os.listdir('../celebrity_images')
for folder in folders:
    filenames = os.listdir('../celebrity_images/{}'.format(folder))
    for filename in filenames:
        try:
            vector = helper('../celebrity_images/{}/{}'.format(folder,filename))
            if vector is not None:
                data['data'].append(vector)
                data['label'].append(folder)
                print('Feature Extracted Sucessfully')
        except:
            pass
        
            

Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Extracted Sucessfully
Feature Ex

In [ ]:
data.keys()

dict_keys(['data', 'label'])

In [ ]:
pd.Series(data['label']).value_counts()

Robert Downey Jr      125
Barack Obama          122
Donald Trump          122
Scarlett Johansson    119
Elon Musk             117
Lionel Messi          114
Roger Federer         111
Cristiano Ronaldo     109
Sachin Tendulkar      107
Salman Khan           107
Leonardo DiCaprio     106
Aamir Khan            103
Angelina Jolie         98
Joe Biden              98
Tom Curise             96
Name: count, dtype: int64

In [ ]:
# save the data 
#pickle.dump(data,open('data_face_features.pickle',mode='wb'))